In [1]:
import numpy as np
import torch
from torch.optim import SGD
import casadi as ca

from modules.modules. import TestModel

### Numpy-Pytorch Memory Sharing

In [2]:
p = np.array([1.])
model = TestModel(p)
optim = SGD(model.parameters())
print(model.p)

x = torch.tensor([1], dtype=torch.float32)

Parameter containing:
tensor([1.], dtype=torch.float64, requires_grad=True)


In [3]:
# in-place operation
p+=1
print(model.p, p)

p[0] = 3
print(model.p, p)

# pytorch optimizer
loss = model(x)
optim.zero_grad()
loss.backward()
optim.step()
print(model.p, p)

Parameter containing:
tensor([2.], dtype=torch.float64, requires_grad=True) [2.]
Parameter containing:
tensor([3.], dtype=torch.float64, requires_grad=True) [3.]
Parameter containing:
tensor([2.9990], dtype=torch.float64, requires_grad=True) [2.999]


### Casadi Derivatives (1D)

In [4]:
# Create symbolic variables
x = ca.SX.sym('x') # ca.MX.sym('x')
y = ca.SX.sym('y') # ca.MX.sym('y')

# Define an expression
f = x**2 + y**2  # Simple example: f(x, y) = x^2 + y^2

# Create a CasADi function from the expression
func = ca.Function('f', [x, y], [f])

# Define the Jacobian of the expression
jacobian = ca.jacobian(f, x)

# Create a function for the Jacobian
jacobian_func = ca.Function('jacobian', [x, y], [jacobian])

# Now, we can evaluate the function and its Jacobian at specific values
x_value = np.array(1.0)
y_value = np.array(2.0)

f_val = func(x_value, y_value)
jacobian_val = jacobian_func(x_value, y_value)

# Output the results
print(f"Function value at ({x_value}, {y_value}): {f_val}")
print(f"Jacobian at ({x_value}, {y_value}):\n{jacobian_val}")


Function value at (1.0, 2.0): 5
Jacobian at (1.0, 2.0):
2


In [5]:
# Batch input values: Let's evaluate for 3 different (x, y) pairs
x_batch = np.array([1.0, 2.0, 3.0])
y_batch = np.array([2.0, 3.0, 4.0])

# Call the function and Jacobian in a batch
f_vals = func(x_batch, y_batch)  # Evaluate f(x, y) for the batch
jacobian_vals = jacobian_func(x_batch, y_batch)  # Evaluate Jacobian for the batch

# Print the results
print("Function values for batch:")
print(f_vals)

print("\nJacobian values for batch:")
print(jacobian_vals)

Function values for batch:
[5, 13, 25]

Jacobian values for batch:
[2, 4, 6]


### Casadi Derivatives (2D)

In [6]:
# Create symbolic variables
x = ca.SX.sym('x', 2) # ca.MX.sym('x')
y = ca.SX.sym('y', 1) # ca.MX.sym('y')

# Define an expression
f = ca.sum1(x)**2 + y**2  # Simple example: f(x, y) = x^2 + y^2

# Create a CasADi function from the expression
func = ca.Function('f', [x, y], [f])

# Define the Jacobian of the expression
jacobian = ca.jacobian(f, x)

# Create a function for the Jacobian
jacobian_func = ca.Function('jacobian', [x, y], [jacobian])

# Now, we can evaluate the function and its Jacobian at specific values
x_value = np.array([1.0, 2.0])
y_value = 2.0

f_val = func(x_value, y_value)
jacobian_val = jacobian_func(x_value, y_value)

# Output the results
print(f"Function value at ({x_value}, {y_value}): {f_val}")
print(f"Jacobian at ({x_value}, {y_value}):\n{jacobian_val}")


Function value at ([1. 2.], 2.0): 13
Jacobian at ([1. 2.], 2.0):
[[6, 6]]


In [7]:
# Batch input values: Let's evaluate for 3 different (x, y) pairs
x_batch = np.array([[1.0, 2.0, 3.0],
                    [2.0, 3.0, 4.0]])
# y_batch = np.array([2.0, 3.0, 4.0])
y_batch = np.array([[2.0, 3.0, 4.0]])

# Call the function and Jacobian in a batch
f_vals = func(x_batch, y_batch)  # Evaluate f(x, y) for the batch
jacobian_vals = jacobian_func(x_batch, y_batch)  # Evaluate Jacobian for the batch

# Print the results
print("Function values for batch:")
print(f_vals)

print("\nJacobian values for batch:")
print(jacobian_vals)

Function values for batch:
[[13, 34, 65]]

Jacobian values for batch:
[[6, 6, 10, 10, 14, 14]]
